In [1]:
import Pkg; Pkg.activate(@__DIR__);

  Activating project at `~/Documents/eth_courses/notebooks/control/invariance`


In [2]:

# Load Dependencies
using Plots
using LazySets
using LinearAlgebra: I
using InvariantSets, Polyhedra, CDDLib
using MathematicalSystems
using ControlSystems

# Quadcopter

In [3]:
# Discrete time model of a quadcopter
A = [1       0       0   0   0   0   0.1     0       0    0       0       0;
      0       1       0   0   0   0   0       0.1     0    0       0       0;
      0       0       1   0   0   0   0       0       0.1  0       0       0;
      0.0488  0       0   1   0   0   0.0016  0       0    0.0992  0       0;
      0      -0.0488  0   0   1   0   0      -0.0016  0    0       0.0992  0;
      0       0       0   0   0   1   0       0       0    0       0       0.0992;
      0       0       0   0   0   0   1       0       0    0       0       0;
      0       0       0   0   0   0   0       1       0    0       0       0;
      0       0       0   0   0   0   0       0       1    0       0       0;
      0.9734  0       0   0   0   0   0.0488  0       0    0.9846  0       0;
      0      -0.9734  0   0   0   0   0      -0.0488  0    0       0.9846  0;
      0       0       0   0   0   0   0       0       0    0       0       0.9846] 
B = [0      -0.0726  0       0.0726;
     -0.0726  0       0.0726  0;
     -0.0152  0.0152 -0.0152  0.0152;
      0      -0.0006 -0.0000  0.0006;
      0.0006  0      -0.0006  0;
      0.0106  0.0106  0.0106  0.0106;
      0      -1.4512  0       1.4512;
     -1.4512  0       1.4512  0;
     -0.3049  0.3049 -0.3049  0.3049;
      0      -0.0236  0       0.0236;
      0.0236  0      -0.0236  0;
      0.2107  0.2107  0.2107  0.2107] 

C = ones(12)


nx = 12
nu = 4
    

4

## Controller

In [4]:
Q = I(nx)
R = I(nu) * 1000
K = lqr(Discrete,A,B,Q,R)

4×12 Matrix{Float64}:
 -5.09962e-14  -0.286741     -0.0153056  …   0.0334519    0.0468176
 -0.286741     -1.96843e-14   0.0153056      3.76281e-15  0.0468176
 -3.04239e-14   0.286741     -0.0153056     -0.0334519    0.0468176
  0.286741      1.16735e-14   0.0153056     -2.32078e-15  0.0468176

## Compute control invariant set

In [5]:
# Half-space representation
X = Hyperrectangle(low=ones(nx)*-1, high=ones(nx))
U = Hyperrectangle(low=ones(nu)*-1, high=ones(nu))

autSys = @system x⁺ = A*x  x∈X
ctrlSys = @system x⁺ = A*x + B*u x∈X u∈U

MIL = maximum_invariant_set(ctrlSys, K)

Convergence at iteration : 34


HPolytope{Float64, Vector{Float64}}(LazySets.HalfSpace{Float64, Vector{Float64}}[LazySets.HalfSpace{Float64, Vector{Float64}}([1.4894195331916327e-13, 2.040397697840599e-14, -5.63590119169587e-16, 2.668263811403526e-14, -5.72812298477516e-17, 0.6121105490059671, 4.093995717839787e-14, 2.4798559409171865e-15, -7.354903912292069e-16, 2.703076503344348e-14, -5.014890628410855e-16, 1.1842337856628466], 1.0), LazySets.HalfSpace{Float64, Vector{Float64}}([-1.4894195331916327e-13, -2.040397697840599e-14, 5.63590119169587e-16, -2.668263811403526e-14, 5.72812298477516e-17, -0.6121105490059671, -4.093995717839787e-14, -2.4798559409171865e-15, 7.354903912292069e-16, -2.703076503344348e-14, 5.014890628410855e-16, -1.1842337856628466], 1.0), LazySets.HalfSpace{Float64, Vector{Float64}}([1.4716533291330148e-13, 2.077959712046758e-14, -5.469611443405493e-16, 2.6354780856116428e-14, -8.509428054661565e-17, 0.6280440002753512, 4.041193782066253e-14, 2.5525248866440163e-15, -7.050555031581402e-16, 2.671

In [19]:
MIL.constraints[134].a

12-element Vector{Float64}:
  7.63503192956167e-15
 -3.16554942365047
 -8.965461466340162e-15
 -1.1550585691324856e-14
 -0.24364149043438768
 -4.004144118266519e-14
  4.2733659930934946e-14
 -0.7354144413070689
 -7.64660416525997e-15
 -7.9975534257469e-15
  0.37311294461395733
 -9.55505799931698e-14